# SPEECH EMOTION RECOGNITION ( SER )

Contexte:
Reconnaissance des émotions vocales des clients à partir de la parole, dans les centres d'appels, afin d'améliorer le service.

SER = reconnaître les émotions humaines et les états affectifs de la parole, ( en effet, la voix reflète souvent l'émotion sous-jacente à travers le ton et la hauteur. C'est également le phénomène que les animaux comme les chiens et les chevaux utilisent pour comprendre les émotions humaines). 

SER est difficile car les émotions sont subjectives et l'annotation audio est difficile.

LIBROSA =  bibliothèque Python pour analyser l'audio et la musique.

Reconnaissance des émotions de la parole:
Objectif:Construire un modèle pour reconnaître l'émotion de la parole en utilisant les bibliothèques LIBROSA, SKLEARN et le jeu de données RAVDESS.

Reconnaissance des émotions vocales 
nous utiliserons les bibliothèques LIBROSA, SOUNDFILE et SKLEARN (entre autres) pour construire un modèle à l'aide d'un MLPClassifier. Cela sera capable de reconnaître l'émotion des fichiers sonores.
1- Nous allons charger les données, 
2- en extraire les caractéristiques,
3- puis diviser l'ensemble de données en ensembles d'entraînement et de test. 
4- Ensuite, nous initialiserons un MLPClassifier et formerons le modèle. 
5- Enfin, nous calculerons la précision de notre modèle.

# L'ensemble de données
Pour ce mini projet Python, nous utiliserons le jeu de données RAVDESS; il s'agit de l'ensemble de données Ryerson Audio-Visual Database of Emotional Speech and Song. 

Cet ensemble de données contient 7356 fichiers évalués par 247 personnes 10 fois sur la validité émotionnelle, l'intensité et l'authenticité. 

L'ensemble de données est de 24,8 Go provenant de 24 acteurs, mais nous avons réduit le taux d'échantillonnage sur tous les fichiers, et vous pouvez le téléchargeable ici: https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view

In [93]:
# 1- Installation des bibliothèques LIBROSA, NUMPY, SKLEARN, PYAUDIO

In [94]:
conda install librosa numpy sklearn pyaudio


Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - librosa
  - sklearn

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [95]:
# 2- Importation des packages:

In [96]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

 3- Définissons les fonctions:
 
1- extract_feature = pour extraire les fonctionnalités mfcc, chroma et mel d'un fichier son. Cette fonction prend   4 paramètres (le nom du fichier et trois paramètres booléens pour les trois fonctionnalités). 

mfcc = Mel Frequency Cepstral Coefficient, représente le spectre de puissance à court terme d'un son
chroma = concerne les 12 classes de hauteur différentes
mel =  Fréquence du spectrogramme Mel

In [97]:
# 3- DataFlair - Extrait des fonctionnalités (mfcc, chroma, mel) d'un fichier son

In [98]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [99]:
# 4-  Maintenant, définissons un dictionnaire pour contenir les nombres et les émotions disponibles dans le jeu 
#   de données RAVDESS, et une liste pour contenir ceux que nous voulons - calmes, heureux, craintifs, dégoûtants.

In [100]:
# DataFlair - Émotions dans le jeu de données RAVDESS

In [101]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

In [102]:
# DataFlair - Émotions à observer

In [103]:
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

5- Maintenant, chargeons les données avec une fonction load_data () qui prend en paramètre la taille relative de l'ensemble de test. x et y sont des listes vides; nous utiliserons la fonction glob () du module glob pour obtenir tous les chemins d'accès aux fichiers audio de notre jeu de données. 

Donc, pour chacun de ces chemins, obtenons le nom de base du fichier, l'émotion en divisant le nom autour de '-' et en extrayant la troisième valeur:

En utilisant notre dictionnaire d'émotions, ce nombre est transformé en émotion, et notre fonction vérifie si cette émotion est dans notre liste d'émotions observées; sinon, il passe au fichier suivant. Il fait un appel à extract_feature et stocke ce qui est retourné dans 'feature'. Ensuite, il ajoute la fonctionnalité à x et l'émotion à y. Ainsi, la liste x contient les caractéristiques et y contient les émotions. Nous appelons la fonction train_test_split avec ceux-ci, la taille du test et une valeur d'état aléatoire, et la retournons a ala fin.

In [104]:
# 6- DataFlair - Chargement des données et extraction des fonctionnalités pour chaque fichier son

In [129]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/home/le/Documents/Python_Env/SPEECH EMOTION RECOGNITION/DataFlair/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [130]:
# 7- Il est temps de diviser l'ensemble de données en ensembles de formation et de test! Gardons 
#  l'ensemble de test à 25% de tout et utilisons la fonction load_data pour cela.

In [131]:
# #DataFlair - Diviser le jeu de données

In [132]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [133]:
# 8- Observation de la forme des ensembles de données d'entraînement et de test:
# DataFlair - Obtention de la forme des ensembles de données d'entraînement et de test

In [134]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [135]:
# 8- obtenons le nombre d'entités extraites.
# DataFlair - Obtenez le nombre de fonctionnalités extraites

In [136]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


9- Maintenant, initialisons un MLPClassifier. Il s'agit d'un classificateur Perceptron multicouche; il optimise la fonction de perte logarithmique en utilisant LBFGS ou la descente de gradient stochastique. Contrairement à SVM ou Naive Bayes , le MLPClassifier dispose d'un réseau neuronal interne à des fins de classification. Il s'agit d'un modèle ANN anticipé.

In [137]:
# DataFlair - Initialiser le classificateur Perceptron multicouche

In [138]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [139]:
# Ajustons et entraînez le modèle.
# #DataFlair - Formation du modèle

In [140]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [141]:
# Prédisons les valeurs de l'ensemble de test. Cela nous donne y_pred (les émotions prédites pour les
# fonctionnalités de l'ensemble de test).

# DataFlair - Prédire pour l'ensemble de test

In [142]:
y_pred=model.predict(x_test)

Pour calculer la précision de notre modèle, nous allons appeler la fonction precision_score () que nous avons importée de sklearn . Enfin, nous arrondirons la précision à 2 décimales et l'afficherons.

In [143]:
#DataFlair - Calculez la précision de notre modèle

In [144]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

In [145]:
# DataFlair - Imprimer la précision

In [146]:
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 71.88%


# Résumé
Dans ce mini projet Python, nous avons appris à reconnaître les émotions de la parole. 
Nous avons utilisé un MLPClassifier pour cela et avons utilisé la bibliothèque de fichiers sonores pour lire le fichier audio et la bibliothèque librosa pour en extraire des fonctionnalités.

Comme nous le voyons, le modèle a fourni une précision de 71,88%. C'est encore assez bon pour nous.